In [54]:
from pynq.overlays.h264 import H264Overlay

In [55]:
overlay = H264Overlay("h264.bit")

load ready


In [56]:
from pprint import pprint
pprint(overlay.ip_dict)

{'h264enc_with_axi_0': {'addr_range': 65536,
                        'driver': <class 'pynq.overlay.DefaultIP'>,
                        'fullpath': 'h264enc_with_axi_0',
                        'gpio': {},
                        'interrupts': {},
                        'phys_addr': 1136656384,
                        'state': None,
                        'type': 'xilinx.com:user:h264enc_with_axi:1.0'}}


In [57]:
print(hex(overlay.ip_dict['h264enc_with_axi_0']['phys_addr']))

0x43c00000


In [25]:
from pynq import Xlnk

xlnk = Xlnk()

In [26]:
import numpy as np
data_size = int(640*480*1.5)
cma_send = xlnk.cma_array([1, data_size], dtype=np.uint8)

In [38]:
with open('h264.yuv', 'rb') as f:
    image_data = f.read()

In [45]:
for i in range(data_size):
    cma_send[0][i] = image_data[i]

In [69]:
for i in range(10):
    print(hex(cma_send[0][i]))

0x80
0x80
0x80
0x80
0x80
0x80
0x80
0x80
0x7e
0x7e


In [70]:
hex(cma_send.physical_address)

'0x16900000'

In [71]:
cma_recv = xlnk.cma_array([1, data_size], dtype=np.uint8)

In [72]:
hex(cma_recv.physical_address)

'0x16a00000'

In [66]:
from pynq import MMIO

h264 = MMIO(overlay.ip_dict['h264enc_with_axi_0']['phys_addr'], 0x10000)

In [73]:
h264.write(0x04, 0x00001d27)
h264.write(0x08, 0)
h264.write(0x40, 0x00008816)
h264.write(0x14, cma_send.physical_address)
h264.write(0x18, cma_recv.physical_address)
h264.write(0x00, 1)

In [76]:
for i in range(data_size):
    print(hex(cma_recv[0][i]))

0xff
0xf8
0xfa
0xfe
0xbd
0x5a
0x13
0xc9
0xe7
0x16


In [77]:
print(h264.read(0x1c))